In [1]:
from google.colab import drive
drive.mount('/content/drive')
#ls drive/MyDrive/chat_api/dataset.xlsx

Mounted at /content/drive


# Install libraries

In [2]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3be3befce4f2cdc0f80d773e16d840eedf0dbc92d67ebc9ce253e951be3d51b5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [4]:
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [5]:
!pip install bert-score

# Read data and calculate metrics

Russian language tasks

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import pandas as pd
from bert_score import score as bert_score
from rouge_score import rouge_scorer
import numpy as np
import torch

# Загружаем данные
etalon = pd.read_excel('drive/MyDrive/chat_api/dataset_news/dataset.xlsx')
mistral_english = pd.read_csv('drive/MyDrive/chat_api/dataset_news/mistral_russian.csv')
mistral_english = mistral_english[['task', 'solution']]

# Подготавливаем данные для сравнения
mistral_english_dif = etalon.join(mistral_english)
mistral_english_dif = mistral_english_dif[['Section(Russian)', 'Question(Russian)', 'Solution(Russian)', 'Options(Russian)', 'task', 'solution']].rename(
                            columns={'Section(Russian)': 'section',
                                     'Question(Russian)': 'question_etalon',
                                     'Solution(Russian)': 'solution_etalon',
                                     'Options(Russian)': 'option_etalon',
                                     'solution':'solution_llm',
                                     'task':'question_llm'})
# Настраиваем GPU для модели BERTScore
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Подготавливаем расчет метрик
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

# Функция для пакетного расчета BERTScore и ROUGE
def calculate_metrics_batch(dataset, batch_size=16):
    bert_scores_f1 = []
    rouge1_f1 = []
    rouge2_f1 = []

    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]

        # BERTScore
        _, _, F1 = bert_score(
            batch['solution_llm'].tolist(),
            batch['solution_etalon'].tolist(),
            lang="ru", device=device
        )
        bert_scores_f1.extend(F1.tolist())

        # ROUGE
        for _, row in batch.iterrows():
            rouge_scores = scorer.score(row['solution_etalon'], row['solution_llm'])
            rouge1_f1.append(rouge_scores['rouge1'].fmeasure)
            rouge2_f1.append(rouge_scores['rouge2'].fmeasure)

    dataset['BERTScore_F1'] = bert_scores_f1
    dataset['ROUGE1_F1'] = rouge1_f1
    dataset['ROUGE2_F1'] = rouge2_f1
    return dataset

# Применяем расчет метрик пакетами
mistral_english_dif_with_metrics = calculate_metrics_batch(mistral_english_dif)

# Выводим результаты
print("Gemini Results:")
print(mistral_english_dif_with_metrics)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Gemini Results:
     section                                    question_etalon  \
0    Алгебра          Каким числом оканчавается выражение 9^121   
1     Логика  Какое число соответсвует вопросительному знаку...   
2     Логика  Какое число должно быть вместо вопросительного...   
3    Алгебра  Среднее арифметическое шести чисел равно 70, а...   
4     Логика  Замените буквы цифрами так, чтобы результат сл...   
..       ...                                                ...   
906   Логика  В коробке есть три красных, 3 желтых, 1 зелены...   
907   Логика  В коробке есть три красных, 3 желтых, 1 зелены...   
908   Логика  В слове МАТЕМАТИКА стерли 6 букв (возможно, ср...   
909   Логика  Восстановив порядок букв в каждой группе, вы п...   
910   Логика  Восстановив порядок букв в каждой группе, вы п...   

                                       solution_etalon  \
0    Начнем с того, что 9^1 = 9, 9^2 = 81, 9^3 = 72...   
1    Закономерность данного ряда следующая: к перво...   
2    

In [ ]:
mistral_english_dif_with_metrics.head()

,section,question_etalon,solution_etalon,option_etalon,question_llm,solution_llm,BERTScore_F1,ROUGE1_F1,ROUGE2_F1
0,Алгебра,Каким числом оканчавается выражение 9^121,"Начнем с того, что 9^1 = 9, 9^2 = 81, 9^3 = 72...",A) 1 B) 7 C) 3 D) 9 E) 5,Каким числом оканчавается выражение 9^121,"Решение:\n\nЧтобы определить, каким числом ок...",0.752710,0.615385,0.507937
1,Логика,Какое число соответсвует вопросительному знаку...,Закономерность данного ряда следующая: к перво...,A) 155 B) 75 C) 53 D) 99 E) 57,Какое число соответсвует вопросительному знаку...,"Решение:\n\nЗадача состоит в том, чтобы найти...",0.633443,0.297872,0.043478
2,Логика,Какое число должно быть вместо вопросительного...,Для выполнения данного задания необходимо допи...,A) 15 B) 25 C) 33 D) 45 E) 43,Какое число должно быть вместо вопросительного...,"Для решения этой задачи необходимо понимать, ...",0.629921,0.347826,0.089552
3,Алгебра,"Среднее арифметическое шести чисел равно 70, а...","Пусть сумма шести чисел равна S1, а сумма четы...",A) 85 B) 82 C) 17 D) 14 E) 36,"Среднее арифметическое шести чисел равно 70, а...",Решение: Чтобы найти среднее арифметическое в...,0.742007,0.550000,0.263158
4,Логика,"Замените буквы цифрами так, чтобы результат сл...","Слагаемые - числа четырёхзначные, а сумма - чи...",A) произведение различных цифр кратно 120 B) п...,"Замените буквы цифрами так, чтобы результат сл...","Для решения этой задачи, необходимо найти циф...",0.662680,0.086957,0.000000


In [ ]:
#save1 = mistral_english_dif_with_metrics
#save1.to_excel('drive/MyDrive/chat_api/metrics_corr/mistral_ru_results.xlsx', index=False)

In [8]:
from transformers import logging
logging.set_verbosity_error()

English language tasks

In [9]:
import pandas as pd
from bert_score import score as bert_score
from rouge_score import rouge_scorer
import numpy as np
import torch

# Загружаем данные
etalon = pd.read_excel('drive/MyDrive/chat_api/dataset_news/dataset.xlsx')
mistral_english = pd.read_csv('drive/MyDrive/chat_api/dataset_news/mistral_english.csv')
mistral_english = mistral_english[['task', 'solution']]

# Подготавливаем данные для сравнения
mistral_english_dif = etalon.join(mistral_english)
mistral_english_dif = mistral_english_dif[['Section(English)', 'Question(English)', 'Solution(English)', 'Options(Russian)', 'task', 'solution']].rename(
                            columns={'Section(English)': 'section',
                                     'Question(English)': 'question_etalon',
                                     'Solution(English)': 'solution_etalon',
                                     'Options(Russian)': 'option_etalon',
                                     'solution':'solution_llm',
                                     'task':'question_llm'})
# Настраиваем GPU для модели BERTScore
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Подготавливаем расчет метрик
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

# Функция для пакетного расчета BERTScore и ROUGE
def calculate_metrics_batch(dataset, batch_size=16):
    bert_scores_f1 = []
    rouge1_f1 = []
    rouge2_f1 = []

    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]

        # BERTScore
        _, _, F1 = bert_score(
            batch['solution_llm'].tolist(),
            batch['solution_etalon'].tolist(),
            lang="en", device=device
        )
        bert_scores_f1.extend(F1.tolist())

        # ROUGE
        for _, row in batch.iterrows():
            rouge_scores = scorer.score(row['solution_etalon'], row['solution_llm'])
            rouge1_f1.append(rouge_scores['rouge1'].fmeasure)
            rouge2_f1.append(rouge_scores['rouge2'].fmeasure)

    dataset['BERTScore_F1'] = bert_scores_f1
    dataset['ROUGE1_F1'] = rouge1_f1
    dataset['ROUGE2_F1'] = rouge2_f1
    return dataset

# Применяем расчет метрик пакетами
mistral_english_dif_with_metrics = calculate_metrics_batch(mistral_english_dif)

# Выводим результаты
print("Gemini Results:")
print(mistral_english_dif_with_metrics)

Gemini Results:
     section                                    question_etalon  \
0    Algebra  What is the number that ends the expression 9^121   
1      Logic  What number corresponds to the question mark: ...   
2      Logic  What number should be in place of the question...   
3    Algebra  The Algebra mean of six numbers is 70 and the ...   
4      Logic  Replace the letters with digits so that the re...   
..       ...                                                ...   
906    Logic  There are three red, 3 yellow, 1 green and 2 b...   
907    Logic  There are three red, 3 yellow, 1 green and 2 b...   
908    Logic  In the word MATEMATIKA, 6 letters were erased ...   
909    Logic  Having restored the order of the letters in ea...   
910    Logic  Having restored the order of the letters in ea...   

                                       solution_etalon  \
0    Let's start with the fact that 9^1 = 9, 9^2 = ...   
1    The pattern of this series is as follows: 2 is...   
2    

In [ ]:
#save1 = mistral_english_dif_with_metrics
#save1.to_excel('drive/MyDrive/chat_api/metrics_corr/mistral_en_results.xlsx', index=False)